In [6]:
import os
import re
import hgtk # 자음 모음 관리
import pandas as pd
from konlpy.tag import Mecab, Okt

compose_code = "\u33c6"

def _jamo_sentence(sent):
    doublespace_pattern = re.compile(r"\s+")

    def __transform(char):
        if char == " ":
            return char
        cjj = hgtk.text.decompose(char, compose_code)
        if not cjj:
            return char
        if len(cjj) == 1:
            return cjj
        cjj_ = "".join(c if c != " " else compose_code for c in cjj)
        return cjj_

    sent_ = "".join(__transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(" ", sent_) # 두칸 이상 공백을 1칸 공백으로
    return sent_

In [7]:
doc="문서"
_jamo_sentence(doc)

'ㅁㅜㄴᴥㅅㅓᴥ'

In [26]:
document = pd.read_csv("natepann.csv")
document = document["제목"].apply(str)+" "+document["본문"].apply(str)

print(document)
for i in range(len(document)):
    document[i] = re.sub('[^ ㄱ-ㅣ가-힣]','',document[i])
    
print(document)

for i in range(len(document)):
    document[i] = _jamo_sentence(document[i])

print(document)
document.to_csv('natepann.txt', sep = '\t', index = False)

0                                     히틀러가 대중들을 다루던 방법  
1      니들이 태용 얼굴 깐다고 태용이 못생겨 질거같음 니들이 평생 이태용 까면서 발작해도...
2      아이즈원 멤버들 근황 권은비팔로우미 최근에 솔로 데뷔 앨범 마무리사쿠라김채원과 함께...
3      나 모르는 동네에 버려 짐 도와줘 어 떡해 엄마가 만원 찾아놓은 거 가방에 넣어두라...
4      남편이 저보고 식 탐 있다는 데 제가 식 탐인지 봐주세요 거두절미 하고 제목  처럼...
                             ...                        
545    정말 결혼 생활이 행복한 분 있나요 판에는 결혼에 부정적인 글이 많지만읽는 분들 중...
546    도련님에게 존댓말 쓰고 처남에겐 반말해도 되나요 오늘 같이 식사하는데 엄마가 남편에...
547                                     일본에 생긴 한국형 편의점  
548     로제 팬들이 자기 노래 불러주니까 울었대 중간에 놀라면서  표정짔는거  링크 걸어둘게 
549    설현이 마른 거냐 제목 살짝 어그로고 설현 당연히 말랐음근데 궁금한게어떻게 말랐는데...
Length: 550, dtype: object
0                                     히틀러가 대중들을 다루던 방법  
1      니들이 태용 얼굴 깐다고 태용이 못생겨 질거같음 니들이 평생 이태용 까면서 발작해도...
2      아이즈원 멤버들 근황 권은비팔로우미 최근에 솔로 데뷔 앨범 마무리사쿠라김채원과 함께...
3      나 모르는 동네에 버려 짐 도와줘 어 떡해 엄마가 만원 찾아놓은 거 가방에 넣어두라...
4      남편이 저보고 식 탐 있다는 데 제가 식 탐인지 봐주세요 거두절미 하고 제목  처럼...
                             ...                        
545 

In [28]:
import datetime
import time
import fasttext

save_fname = 'natepann.txt'
model_fname = 'result.bin'
start_time = time.time()

print("학습 중")
model = fasttext.train_unsupervised(
    input=save_fname,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.001,
    dim=300,
    epoch=5,
    min_count=10,
    thread=25,
    verbose=1,
)

model.save_model(model_fname)

print('완료')
print("time : ", time.time() - start_time)

학습 중
완료
time :  57.92689800262451


In [29]:
start_time = time.time()
model_fname = 'result.bin'

model = fasttext.load_model(model_fname)
print("time : ", time.time() - start_time)

time :  2.6297528743743896


In [35]:
model.get_nearest_neighbors("ㅁㅜㄴᴥㅅㅓᴥ")

[(0.48024001717567444, 'ㅎㅏᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.47022518515586853, 'ㄱㅏᴥㅅㅓᴥ'),
 (0.4551432728767395, 'ㅎㅐᴥㅅㅓᴥ'),
 (0.45377427339553833, 'ㄱㅡᴥㄹㅐᴥㅅㅓᴥ'),
 (0.4498012065887451, 'ㅇㅘᴥㅅㅓᴥ'),
 (0.44788995385169983, 'ㅅㅣㅍᴥㅇㅓᴥㅅㅓᴥ'),
 (0.4477541446685791, 'ㅇㅣㅆᴥㅇㅓᴥㅅㅓᴥ'),
 (0.4375804662704468, 'ㅇㅏㄶᴥㅇㅏᴥㅅㅓᴥ'),
 (0.43746110796928406, 'ㄱㅏㅌᴥㅇㅏᴥㅅㅓᴥ'),
 (0.4258884787559509, 'ㅈㅓᴥㄴㅡㄴᴥ')]

In [33]:
hgtk.text.compose("ㅁㅜㄴᴥㅅㅓᴥ")

'문서'

In [48]:
model.get_nearest_neighbors("ㅁㅜㄴᴥㅅㅓᴥ")[0]

(0.48024001717567444, 'ㅎㅏᴥㅁㅕㄴᴥㅅㅓᴥ')

In [49]:
[word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))]

['ㅎㅏᴥㅁㅕㄴᴥㅅㅓᴥ',
 'ㄱㅏᴥㅅㅓᴥ',
 'ㅎㅐᴥㅅㅓᴥ',
 'ㄱㅡᴥㄹㅐᴥㅅㅓᴥ',
 'ㅇㅘᴥㅅㅓᴥ',
 'ㅅㅣㅍᴥㅇㅓᴥㅅㅓᴥ',
 'ㅇㅣㅆᴥㅇㅓᴥㅅㅓᴥ',
 'ㅇㅏㄶᴥㅇㅏᴥㅅㅓᴥ',
 'ㄱㅏㅌᴥㅇㅏᴥㅅㅓᴥ',
 'ㅈㅓᴥㄴㅡㄴᴥ']

In [50]:
[word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))][0]

'ㅎㅏᴥㅁㅕㄴᴥㅅㅓᴥ'

In [51]:
type([word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))])

list

In [54]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))]]

['하면서', '가서', '해서', '그래서', '와서', '싶어서', '있어서', '않아서', '같아서', '저는']

In [56]:
###############################################################################

In [57]:
document = pd.read_csv("natepann.csv")
document = document["제목"].apply(str)+" "+document["본문"].apply(str)

print(document)
for i in range(len(document)):
    document[i] = re.sub('[^ ㄱ-ㅣ가-힣]','',document[i])
    
print(document)

for i in range(len(document)):
    document[i] = _jamo_sentence(document[i])

print(document)

document.to_csv('natepann.txt', sep = '\t', index = False)

0        이나은 드디어 터졌네썰 진짜 속시원하다ㅋㅋㅋ 혹시 몰라서 나도 썰 하나 풀게난 전에...
1                          진짜 이나은 수준 무슨일이냐 할말이 있고 못할말이 있지 
2        아이들이 살아나는 방법 우선 인증가능 타팬임 그냥 아이들 무대만 가끔 보는 정도물론...
3              에이프릴에 이렇게 예쁜 멤버가 있었음 저 얼굴을 안밀어줬네이나은밖에 몰랐는데 
4                                       와 이 부분 보고 ㅈㄴ 울컥함  
                               ...                        
10695    정말 결혼 생활이 행복한 분 있나요 판에는 결혼에 부정적인 글이 많지만읽는 분들 중...
10696    도련님에게 존댓말 쓰고 처남에겐 반말해도 되나요 오늘 같이 식사하는데 엄마가 남편에...
10697                                     일본에 생긴 한국형 편의점  
10698    로제 팬들이 자기노래 불러주니까 울었대 중간에 놀라면서 ㅇㅅㅇ 표정짔는거 ㄱㅇㅇ 링...
10699    설현이 마른거냐ㅋ 제목 살짝 어그로고 설현 당연히 말랐음근데 궁금한게어떻게 말랐는데...
Length: 10700, dtype: object
0        이나은 드디어 터졌네썰 진짜 속시원하다ㅋㅋㅋ 혹시 몰라서 나도 썰 하나 풀게난 전에...
1                          진짜 이나은 수준 무슨일이냐 할말이 있고 못할말이 있지 
2        아이들이 살아나는 방법 우선 인증가능 타팬임 그냥 아이들 무대만 가끔 보는 정도물론...
3              에이프릴에 이렇게 예쁜 멤버가 있었음 저 얼굴을 안밀어줬네이나은밖에 몰랐는데 
4                                       와 이 부분 보고 ㅈㄴ 울컥함  
                           

In [58]:
import datetime
import time
import fasttext

save_fname = 'natepann.txt'
model_fname = 'result.bin'
start_time = time.time()

print("학습 중")
model = fasttext.train_unsupervised(
    input=save_fname,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.001,
    dim=300,
    epoch=5,
    min_count=10,
    thread=25,
    verbose=1,
)

model.save_model(model_fname)

print('완료')
print("time : ", time.time() - start_time)

학습 중
완료
time :  163.41144180297852


In [59]:
start_time = time.time()
model_fname = 'result.bin'

model = fasttext.load_model(model_fname)
print("time : ", time.time() - start_time)

time :  2.885312795639038


In [60]:
model.get_nearest_neighbors("ㅁㅜㄴᴥㅅㅓᴥ")

[(0.9999961853027344, 'ㅎㅣㅁᴥㄷㅡㄹᴥㅇㅓᴥㅅㅓᴥ'),
 (0.9999958872795105, 'ㅅㅜㄴᴥㅅㅓᴥ'),
 (0.9999958872795105, 'ㅈㅜㄴᴥㅂㅣᴥㅎㅏᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999957084655762, 'ㄸㅓㄹᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999955296516418, 'ㅇㅜㄹᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999952912330627, 'ㅎㅏㄱᴥㄱㅛᴥㅇㅔᴥㅅㅓᴥ'),
 (0.9999951124191284, 'ㄴㅐᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999949932098389, 'ㅎㅗㄴᴥㅈㅏᴥㅅㅓᴥ'),
 (0.9999948143959045, 'ㅂㅗᴥㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999947547912598, 'ㄴㅐᴥㅇㅔᴥㅅㅓᴥ')]

In [61]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))]]

['힘들어서', '순서', '준비하면서', '떨면서', '울면서', '학교에서', '내면서', '혼자서', '보면서', '내에서']

In [62]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("스우파"))]]

['스테이', '가스라이팅', '스토리', '에이프릴', '하이라이트', '마인드', '인스타그램', '와이프와', '네이버', '하이브']

In [63]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("댕댕이"))]]

['신랑이', '이상이', '집주인이', '없이', '본인이', '점이', '사진이', '등이', '아린이', '걱정이']

In [64]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("멍멍이"))]]

['표정이', '이상이', '명이', '엉덩이', '없이', '사진이', '걱정이', '정이', '형이', '과정이']

In [65]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("냉이"))]]

['형이', '성인이', '없이', '등이', '인생이', '진이', '사진이', '명이', '인성이', '지인이']

In [66]:
import datetime
import time
import fasttext

save_fname = 'natepann.txt'
model_fname = 'result2.bin'
start_time = time.time()

print("학습 중")
model = fasttext.train_unsupervised(
    input=save_fname,
    model='cbow',
    loss='hs',
    ws=20,
    lr=0.001,
    dim=300,
    epoch=5,
    min_count=5,
    thread=25,
    verbose=1,
)

model.save_model(model_fname)

print('완료')
print("time : ", time.time() - start_time)

학습 중
완료
time :  233.84615683555603


In [67]:
start_time = time.time()
model_fname = 'result2.bin'

model = fasttext.load_model(model_fname)
print("time : ", time.time() - start_time)

time :  2.6841368675231934


In [68]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("댕댕이"))]]

['등이', '인성이', '정이', '인원이', '엉덩이', '대상이', '형이', '명이', '정상이', '경향이']

In [69]:
############################################################################### 명사 단위 csv

In [85]:
import pandas as pd
import re
from konlpy.tag import Okt

okt = Okt() 

titles = []
txt = []
title_morphs = []
txt_morphs = []

document = pd.read_csv("natepann.csv")
document = document["제목"].apply(str)+" "+document["본문"].apply(str)

print(document)
for i in range(len(document)):
    document[i] = okt.nouns(document[i])
    
print(document)
for i in range(len(document)):
    document[i] = ' '.join(document[i])
    
print(document)
for i in range(len(document)):
    document[i] = _jamo_sentence(document[i])

print(document)

document.to_csv('natepann_nouns.txt', sep = '\t', index = False, encoding="utf-8-sig")

#명사

0        이나은 드디어 터졌네썰 진짜 속시원하다ㅋㅋㅋ 혹시 몰라서 나도 썰 하나 풀게난 전에...
1                          진짜 이나은 수준 무슨일이냐 할말이 있고 못할말이 있지 
2        아이들이 살아나는 방법 우선 인증가능 타팬임 그냥 아이들 무대만 가끔 보는 정도물론...
3              에이프릴에 이렇게 예쁜 멤버가 있었음 저 얼굴을 안밀어줬네이나은밖에 몰랐는데 
4                                       와 이 부분 보고 ㅈㄴ 울컥함  
                               ...                        
10695    정말 결혼 생활이 행복한 분 있나요 판에는 결혼에 부정적인 글이 많지만읽는 분들 중...
10696    도련님에게 존댓말 쓰고 처남에겐 반말해도 되나요 오늘 같이 식사하는데 엄마가 남편에...
10697                                     일본에 생긴 한국형 편의점  
10698    로제 팬들이 자기노래 불러주니까 울었대 중간에 놀라면서 ㅇㅅㅇ 표정짔는거 ㄱㅇㅇ 링...
10699    설현이 마른거냐ㅋ 제목 살짝 어그로고 설현 당연히 말랐음근데 궁금한게어떻게 말랐는데...
Length: 10700, dtype: object
0        [나은, 진짜, 속, 혹시, 하나, 풀, 난, 전, 나은, 둔산, 동창, 사촌오빠,...
1                                         [진짜, 나은, 수준, 일이]
2        [아이, 방법, 우선, 인증, 가능, 타팬, 임, 그냥, 아이, 무대, 가끔, 정도...
3                                    [에이프릴, 멤버, 저, 얼굴, 나은]
4                                           [이, 부분, 보고, 함]
                           

In [86]:
import datetime
import time
import fasttext

save_fname = 'natepann_nouns.txt'
model_fname = 'result2.bin'
start_time = time.time()

print("학습 중")
model = fasttext.train_unsupervised(
    input=save_fname,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.001,
    dim=300,
    epoch=5,
    min_count=10,
    thread=25,
    verbose=1,
)

model.save_model(model_fname)

print('완료')
print("time : ", time.time() - start_time)

학습 중
완료
time :  94.21820783615112


In [87]:
start_time = time.time()
model_fname = 'result2.bin'

model = fasttext.load_model(model_fname)
print("time : ", time.time() - start_time)

time :  2.6360692977905273


In [88]:
model.get_nearest_neighbors("ㅁㅜㄴᴥㅅㅓᴥ")

[(0.9999939203262329, 'ㅈㅜㄴᴥㅅㅓᴥ'),
 (0.9999935030937195, 'ㅅㅜㄴᴥㅅㅓᴥ'),
 (0.9999915361404419, 'ㅁㅜㄴᴥㅅㅣㄴᴥ'),
 (0.9999903440475464, 'ㅁㅕㄴᴥㅅㅓᴥ'),
 (0.9999901056289673, 'ㅇㅏᴥㄴㅏᴥㅇㅜㄴᴥㅅㅓᴥ'),
 (0.9999898672103882, 'ㅅㅏᴥㅅㅓᴥ'),
 (0.9999896883964539, 'ㅁㅜㄴᴥㅈㅏㅇᴥ'),
 (0.9999895691871643, 'ㅁㅜㄴᴥ'),
 (0.99998939037323, 'ㅁㅜㄴᴥㅈㅏᴥ'),
 (0.9999889135360718, 'ㅁㅜㄴᴥㅈㅐᴥㅇㅣㄴᴥ')]

In [89]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("문서"))]]

['준서', '순서', '문신', '면서', '아나운서', '사서', '문장', '문', '문자', '문재인']

In [90]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("스우파"))]]

['우파', '스파', '아이라인', '나인', '사이트', '스라이', '사이', '라인', '브이', '나이']

In [91]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("멍멍이"))]]

['정이', '방이', '중이', '엉덩이', '사이', '투성이', '상이', '이', '판이', '종이']

In [92]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("댕댕이"))]]

['상이', '방이', '종이', '용이', '호랑이', '판이', '이', '정이', '가장이', '일이']

In [93]:
[hgtk.text.compose(found) for found in [word for score, word in model.get_nearest_neighbors(_jamo_sentence("냉이"))]]

['방이', '중이', '사이', '종이', '용이', '길이', '아이라인', '정이', '천이', '만이']